# Generating training instances from genomic dna, intron and mRNA positions.

> Preparing training instances.

## Goals
1. Get the "edge" of the intron and;
    1. position the edge at the center, or forward or backward in a training string
    2. Wherever the "edge" is, replace the gene sequence with the INTRON token on the target string
2. Get the "edge" of the mRNA and;
    1. position the edge at the center, or forward or backward in a training string
    2. Wherever the "edge" is, replace the gene sequence with the NON-mRNA token on the target string
3. Generate training instances that;
    1. Include the "edge" of the intron with various shifting strategies
    2. Exclude the "edge" of the intron and get transcribed mRNA
    3. Include the "edge" of the mRNA with various shifting strategies

In [1]:
#| default_exp training.transcription.generation

## 0. Setup

In [6]:
#| export
from pathlib import Path

from llm_mito_scanner.data.download import load_config, \
    get_latest_assembly_path, get_genomic_genbank_path
from llm_mito_scanner.training.transcription.index import get_intron_locations

In [4]:
#| hide
config = load_config()

In [8]:
#| hide
data_path = Path(config.get("data_path"))
data_raw_path = data_path / "raw"
assemblies_path = data_raw_path / "assemblies"
latest_assembly_path = get_latest_assembly_path(assemblies_path)
genomic_genbank_path = get_genomic_genbank_path(latest_assembly_path)
genes_path = latest_assembly_path / "genes"
training_data_path = latest_assembly_path / "training"
transcription_data_path = training_data_path / "transcription"
intron_locations_path = transcription_data_path / "intron_positions"
for p in [genes_path, intron_locations_path]:
    if not p.exists():
        raise FileNotFoundError(f"This notebook requires the path {p.resolve()} to exist")

In [9]:
#| hide
intron_locations = get_intron_locations(intron_locations_path)
intron_locations.head()

,geneid,transcriptid,intron_start,intron_end,mrna_start,mrna_end,chromosome
0,GeneID:79501,NM_001005484.2,15,101,0,6167,NC_000001.11
1,GeneID:79501,NM_001005484.2,155,3618,0,6167,NC_000001.11
2,GeneID:112268260,XM_047436352.1,186,547,0,17102,NC_000001.11
3,GeneID:112268260,XM_047436352.1,1339,2365,0,17102,NC_000001.11
4,GeneID:112268260,XM_047436352.1,2467,8912,0,17102,NC_000001.11


## 1. Intron Edge Sequence Generation

## 2. mRNA Edge Sequence Generation

## 3. Generate Training Instances

In [10]:
#| hide
import nbdev; nbdev.nbdev_export()